In [2]:
import pandas as pd

In [3]:
# Read the CSV files into DataFrames
df = pd.read_csv("NEW_M_J.csv", parse_dates=True, index_col="datetime")

Merge of the two csv datasets plus a new one, expanding from November 1st 2023 to August 1st 2023 - May 15 2024
Source: https://www.visualcrossing.com/weather/weather-data-services#

In [4]:
df = df[['location', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility',
       'moonphase', 'conditions', 'description']]

# df = df[['location', 'dew', 'humidity', 'precip', 'precipcover', 'windgust', 'cloudcover', 'visibility']]

In [7]:
df_numerical = df[['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility',
       'moonphase']].copy()

The true purpose of regression is to seperate the total variance in lowtemp and hightemp from the variance explained by dew and the variance that is still unexplained.

# Full dataset predictions - not seperated by locations

In [14]:
# Define features and target
columns_to_drop = ['location', 'tempmax', 'tempmin', 'conditions', 'description']
features = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
target_max = df['tempmax']
target_min = df['tempmin']

# Train the model for tempmax on the entire dataset
ridge_max = Ridge()
ridge_max.fit(features, target_max)

# Train the model for tempmin on the entire dataset
ridge_min = Ridge()
ridge_min.fit(features, target_min)

# Predict tempmax and tempmin for the entire dataset
y_pred_max = ridge_max.predict(features)
y_pred_min = ridge_min.predict(features)

# Create the prediction columns if they don't exist
if 'predicted_tempmax' not in df.columns:
    df['predicted_tempmax'] = pd.Series([0.0] * len(df), index=df.index)

if 'predicted_tempmin' not in df.columns:
    df['predicted_tempmin'] = pd.Series([0.0] * len(df), index=df.index)

# Add predictions to the original dataframe
df['predicted_tempmax'] = y_pred_max
df['predicted_tempmin'] = y_pred_min

# Ensure alignment by resetting the indices
df = df.sort_index()

# Display the dataframe with predictions
df[['location', 'tempmax', 'predicted_tempmax', 'tempmin', 'predicted_tempmin']].tail(5)

,location,tempmax,predicted_tempmax,tempmin,predicted_tempmin
datetime,,,,,
2024-07-07,London,18.7,18.789531,10.2,10.560030
2024-07-07,Ottawa,26.4,26.456012,19.5,19.233058
2024-07-07,Berlin,23.4,23.275686,14.9,15.052070
2024-07-07,Mexico City,24.1,23.625810,13.6,13.666405
2024-07-07,Washington DC,31.2,32.956585,24.7,22.823749


In [15]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Verify that the DataFrame has no null values
# assert df.isnull().sum().sum() == 0, "There are null values in the DataFrame"

# # Get the unique locations
# locations = df['location'].unique()

# # Plotting by location and year
# for location in locations:
#     df_location = df[df['location'] == location]
#     years = df_location.index.year.unique()

#     for year in years:
#         df_year = df_location[df_location.index.year == year]

#         plt.figure(figsize=(20, 7))

#         # Plotting tempmax and predicted_tempmax
#         plt.subplot(2, 1, 1)
#         plt.plot(df_year.index, df_year['tempmax'], marker='o', label='Actual Temp Max')
#         plt.plot(df_year.index, df_year['predicted_tempmax'], marker='x', linestyle='--', label='Predicted Temp Max')
#         plt.title(f'Actual vs Predicted Temp Max - {location} ({year})')
#         plt.xlabel('Date')
#         plt.ylabel('Temperature')
#         plt.legend()

#         # Plotting tempmin and predicted_tempmin
#         plt.subplot(2, 1, 2)
#         plt.plot(df_year.index, df_year['tempmin'], marker='o', label='Actual Temp Min')
#         plt.plot(df_year.index, df_year['predicted_tempmin'], marker='x', linestyle='--', label='Predicted Temp Min')
#         plt.title(f'Actual vs Predicted Temp Min - {location} ({year})')
#         plt.xlabel('Date')
#         plt.ylabel('Temperature')
#         plt.legend()

# Full dataset - test MSE and R2

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Reset the index
df.reset_index(drop=True, inplace=True)

# Create a copy of the DataFrame
new_df = df.copy()

# Create the prediction columns if they don't exist
if 'predicted_tempmax' not in new_df.columns:
    new_df['predicted_tempmax'] = pd.Series([0.0] * len(new_df), index=new_df.index)
if 'predicted_tempmin' not in new_df.columns:
    new_df['predicted_tempmin'] = pd.Series([0.0] * len(new_df), index=new_df.index)

# Define features and target
columns_to_drop = ['location', 'tempmax', 'tempmin', 'conditions', 'description']
features = new_df.drop(columns=[col for col in columns_to_drop if col in new_df.columns])
target_max = new_df['tempmax']
target_min = new_df['tempmin']

# Split the data into training and testing sets
X_train, X_test, y_train_max, y_test_max = train_test_split(features, target_max, test_size=0.2, random_state=42)
X_train_min, X_test_min, y_train_min, y_test_min = train_test_split(features, target_min, test_size=0.2, random_state=42)

# Save the original indices of the test sets
test_indices_max = X_test.index
test_indices_min = X_test_min.index

# Train the model for tempmax
ridge_max = Ridge()
ridge_max.fit(X_train, y_train_max)

# Train the model for tempmin
ridge_min = Ridge()
ridge_min.fit(X_train_min, y_train_min)

# Predict tempmax and tempmin for the test set
y_pred_max = ridge_max.predict(X_test)
y_pred_min = ridge_min.predict(X_test_min)

# Align predictions with the original DataFrame using the saved indices
new_df.loc[test_indices_max, 'predicted_tempmax'] = y_pred_max
new_df.loc[test_indices_min, 'predicted_tempmin'] = y_pred_min

# Calculate and print the mean squared error for evaluation
mse_max = mean_squared_error(y_test_max, y_pred_max)
mse_min = mean_squared_error(y_test_min, y_pred_min)

# Calculate and print the R-squared score for evaluation
r2_max = r2_score(y_test_max, y_pred_max)
r2_min = r2_score(y_test_min, y_pred_min)

print(f'Mean Squared Error for tempmax: {mse_max}')
print(f'Mean Squared Error for tempmin: {mse_min}')
print(f'R-squared Score for tempmax: {r2_max}')
print(f'R-squared Score for tempmin: {r2_min}')

# Display the dataframe with predictions
new_df[['tempmax', 'predicted_tempmax', 'tempmin', 'predicted_tempmin']].head(10)


Mean Squared Error for tempmax: 0.46896580716092956
Mean Squared Error for tempmin: 0.42537550299020577
R-squared Score for tempmax: 0.9950731557528979
R-squared Score for tempmin: 0.9933424923568969


,tempmax,predicted_tempmax,tempmin,predicted_tempmin
0,25.2,25.000529,8.0,9.112929
1,17.6,17.530917,11.3,11.474986
2,9.2,8.815058,3.4,3.480235
3,18.9,19.226029,3.4,2.905445
4,15.7,15.391495,10.6,10.955362
5,32.1,31.826240,5.0,4.607418
6,9.9,11.476712,6.8,5.951686
7,19.1,18.981556,14.1,14.106105
8,16.0,16.009703,9.9,9.249776
9,11.1,11.984696,5.1,5.079981
